# Session 2 - Demo 2.2 - Model Reliability & Enhancing LLMs

<a href="https://colab.research.google.com/github/dair-ai/pe-for-llms/blob/main/notebooks/session-2/demo-2.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install chromadb

In [ ]:
# load the libraries
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

# load the environment variables
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

We will be using `text-davinci-003` for all examples in this tutorial. You can easily switch to `gpt-3.5-turbo`, however, note that this model is more expensive and slower. 

In [2]:
# lm instance
llm_text_davinci = OpenAI(model_name='text-davinci-003', temperature=0)

## Counting token usage

We will use `tiktoken`, an open-source tokenizer by OpenAI.

https://github.com/openai/tiktoken

In [3]:
import tiktoken

In [4]:
# load encoding by name
encoding = tiktoken.get_encoding("cl100k_base")

# load the correct encoding by passing the model name
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [5]:
# tokenize text
encoding.encode("I am feeling happy today")

[40, 1097, 8430, 6380, 3432]

In [6]:
# count tokens
len(encoding.encode("I am feeling happy today"))

5

In [7]:
# using gpt-4 model
gpt4_encoding = tiktoken.encoding_for_model("gpt-4")

In [8]:
gpt4_encoding.encode("I am feeling happy today")

[40, 1097, 8430, 6380, 3432]

You can also calculate token usage with LangChain:

In [18]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

In [19]:
llm = OpenAI(model_name="text-davinci-003")

# anything inside the context manager will be tracked
with get_openai_callback() as cb:
    result = llm("Tell me a short story about a robot")
    print(cb)

Tokens Used: 190
	Prompt Tokens: 8
	Completion Tokens: 182
Successful Requests: 1
Total Cost (USD): $0.0038


## Be clear and specific when prompting

In [20]:
global_trending_movies = ["The Suicide Squad", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]


prompt = """
Your task is to recommend movies to a customer. 

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}. 

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't have a movie to recommend, you should respond "Sorry, couldn't find a movie to recommend today.".

Customer: Please recommend a movie based on my interests.
Agent: 
"""

llm_text_davinci(prompt.format(global_trending_movies=global_trending_movies))

"Sorry, couldn't find a movie to recommend today."

An example where the customer provides information about interests:

In [21]:
global_trending_movies = ["The Suicide Squad", "No Time to Die", "Dune",  "Spider-Man: No Way Home", "The French Dispatch", "Black Widow", "Eternals", "The Matrix Resurrections", "West Side Story", "The Many Saints of Newark"]


prompt = """
Your task is to recommends movies to a customer. 

You are responsible to recommend a movie from the top global trending movies from {global_trending_movies}. 

You should refrain from asking users for their preferences and avoid asking for personal information.

If you don't have a movie to recommend, you should respond "Sorry, couldn't find a movie to recommend today.".

Customer: I love super-hero movies. Please recommend a movie based on my interests.
Agent: 
"""

llm_text_davinci(prompt.format(global_trending_movies=global_trending_movies))

"I recommend Spider-Man: No Way Home. It's a highly anticipated sequel to the Marvel Cinematic Universe and is sure to be a hit with fans of super-hero movies."

## Using Delimiters to Distinguish Components of a Prompt

In [22]:
prompt = """
Convert the following code block in the ### <code> ### section to Python:

###
strings2 = Array[]
strings2.push("one")
strings2.push("two")
strings2.push("THREE")
###
"""

In [23]:
IPython.display.Markdown("```python" + llm_text_davinci(prompt) + "\n```")

```python
strings2 = []
strings2.append("one")
strings2.append("two")
strings2.append("THREE")
```

## Specify Output Format

In [24]:
prompt = """
Your task is: given a product description, return the requested information in the section delimited by ### ###. Format the output as a JSON object.

Product Description: Introducing the Nike Air Max 270 React: a comfortable and stylish sneaker that combines two of Nike's best technologies. With a sleek black design and a unique bubble sole, these shoes are perfect for everyday wear.

###
product_name: the name of the product
product_bran: the name of the brand (if any) 
###
"""

IPython.display.Markdown(llm_text_davinci(prompt))


{
  "product_name": "Nike Air Max 270 React",
  "product_brand": "Nike"
}

## Specifying the Length of the output

In [25]:
prompt = """
Your task is: given a customer support email, which is delimited with ###, generate a shorter 1-2 sentence response.

###
Dear [Customer],

We hope this email finds you well. We wanted to update you on the shipping issue you experienced with your recent order. After investigating the issue, we have located your package and it is currently on its way to you. We apologize for any inconvenience this may have caused and thank you for your patience and understanding while we resolved this matter.

Please note that we have taken steps to prevent similar issues from occurring in the future. We have improved our shipping tracking system and are now better equipped to ensure that packages arrive on time and in good condition. We take the quality of our service very seriously and want to ensure that all of our customers have a positive experience when shopping with us.

Once again, we apologize for any inconvenience this may have caused and hope that you will continue to shop with us in the future. If you have any further questions or concerns, please do not hesitate to contact us. We are always here to help and ensure that your shopping experience is a positive one.

Thank you for your understanding.

Best regards,

[Your Name]

Customer Support Team
###
"""

llm_text_davinci(prompt)

'We apologize for the inconvenience caused by the shipping issue with your recent order. We have located your package and it is on its way to you. We have taken steps to prevent similar issues from occurring in the future. Thank you for your understanding.'

## Avoid deviating and constraining the output

Sometimes it helps to be more specific about what output you expect to avoid the model deviating from the main task of interest. 

In [31]:
llm_text_davinci("Recommend a movie for Saturday:")

"\n\nThe Grand Budapest Hotel (2014). This Wes Anderson classic is a delightful comedy-drama set in a European hotel in the 1930s. It follows the adventures of the hotel's concierge, Gustave H, and his loyal lobby boy, Zero Moustafa, as they try to solve a mystery involving a priceless painting. With an all-star cast, including Ralph Fiennes, Adrien Brody, and Willem Dafoe, this is a must-see for any movie fan."

In [30]:
llm_text_davinci("Recommend a movie for Saturday. Just say the movie, no need for explanations!")

'\n\nThe Princess Bride'